## Задание 1
SQL-запрос

В базе данных есть две таблицы - одна с установками наших игр и вторая с событиями прохождения уровней. Используя эти две таблицы необходимо рассчитать среднее количество уровней, пройденное в первые сутки игры, по всем игрокам, в группировке по проектам. Результат вывести в порядке убывания среднего количества пройденных уровней.
Примечание: установкой здесь считается первое открытие приложения, т.е. `install_time` - это по сути время первой сессии.

Структура таблиц:


### Таблица Installs

| id | app_name | Install_time |
|----|----------|--------------|
| 1  | name_1   | 2024-02-01 10:00:00 |
| 2  | name_2   | 2024-02-01 12:00:00 |
| 3  | name_1   | 2024-02-01 14:00:00 |
| 4  | name_2   | 2024-02-01 16:00:00 |
| 5  | name_1   | 2024-02-01 18:00:00 |
| 6  | name_2   | 2024-02-01 20:00:00 |

### Таблица Levels

| user_id | level | event_time          |
|---------|-------|---------------------|
| 1       | 10    | 2024-02-01 10:30:00 |
| 1       | 11    | 2024-02-01 11:00:00 |
| 2       | 5     | 2024-02-01 12:30:00 |
| 3       | 8     | 2024-02-01 14:30:00 |
| 4       | 3     | 2024-02-01 16:30:00 |
| 5       | 6     | 2024-02-01 18:30:00 |
| 6       | 2     | 2024-02-01 20:30:00 |




Ожидаемый результат запроса:


| app_name | avg_levels_first_day |
|----------|----------------------|
| name_1   | 9.0                  |
| name_2   | 4.0                  |



Для начала сгенерируем данные чтобы можно было наглядно и удобно работать с ними, подключимся к серверу с PostgreSQL и подгрузим наши таблицы в БД.


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sqlalchemy import create_engine, text as sql_text
from google.colab import userdata

In [2]:
#con_pg = create_engine(userdata.get('postgre_url'))

In [3]:
con_pg = create_engine('postgresql+psycopg2://wygxcdzu:\
@user_name')

Небольшая функция для наших запросов.

In [4]:
def select_pg(sql):
  with con_pg.connect() as conn:
    return pd.read_sql(sql_text(sql), conn)

Генерируем данные с условием, что в таблице `levels` дата активности нашего пользователя должна быть больше, чем дата регистрации в приложении `install_time`, а также каждая последующая дата обновления уровня выше, чем предыдущая дата получения уровня.

In [5]:
install_list = []
install_time = datetime.strptime("2024-01-01 00:00:00", "%Y-%m-%d %H:%M:%S")

for i in range(1, 501):
    app_name = f'name_{np.random.randint(1, 6)}'
    install_time = install_time + timedelta(hours=np.random.randint(1, 25),
                                            seconds=np.random.randint(1, 3601)
                                           )

    install_list.append({'id': i, 'app_name': app_name,
                         'install_time': install_time})

installs = pd.DataFrame(install_list).sort_values('install_time')

In [6]:
activities = []

for index, row in installs.iterrows():
    num_levels = np.random.randint(1, 41)
    event_time = row['install_time']

    for level in range(1, num_levels + 1):
        event_time = event_time + timedelta(hours=np.random.randint(1, 25),
                                            seconds=np.random.randint(1, 3601)
                                           )
        activities.append({'user_id': row['id'],
                           'level': level, 'event_time': event_time})

levels = pd.DataFrame(activities).sort_values('event_time')

Посмотрим что получились за данные.

In [7]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            500 non-null    int64         
 1   app_name      500 non-null    object        
 2   install_time  500 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 15.6+ KB


In [8]:
levels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10279 entries, 0 to 10278
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     10279 non-null  int64         
 1   level       10279 non-null  int64         
 2   event_time  10279 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(2)
memory usage: 321.2 KB


In [9]:
installs.head()

,id,app_name,install_time
0,1,name_1,2024-01-01 03:56:41
1,2,name_2,2024-01-02 00:39:51
2,3,name_4,2024-01-02 21:25:35
3,4,name_2,2024-01-03 00:24:12
4,5,name_3,2024-01-03 06:21:35


In [10]:
levels.head()

,user_id,level,event_time
0,1,1,2024-01-01 17:50:59
1,1,2,2024-01-01 19:08:50
2,1,3,2024-01-01 21:31:17
12,2,1,2024-01-02 08:06:20
3,1,4,2024-01-02 12:56:02


Зальем таблички в БД.

In [11]:
installs.to_sql('installs', con_pg, index=False, if_exists='replace')
levels.to_sql('levels', con_pg, index=False, if_exists='replace')

279

Проверим как все работает.

In [12]:
sql = '''
select * from levels
'''

In [13]:
select_pg(sql)

,user_id,level,event_time
0,1,1,2024-01-01 17:50:59
1,1,2,2024-01-01 19:08:50
2,1,3,2024-01-01 21:31:17
3,2,1,2024-01-02 08:06:20
4,1,4,2024-01-02 12:56:02
...,...,...,...
10274,500,34,2024-10-14 21:59:14
10275,500,35,2024-10-15 14:28:27
10276,500,36,2024-10-16 02:14:39
10277,500,37,2024-10-16 11:33:34


Теперь сам запрос.

In [14]:
sql = '''
SELECT app_name,
      AVG(levels) AS average_levels
FROM (
  SELECT user_id,
        app_name,
        ROUND(SUM(level), 2) AS levels,
        DATE_TRUNC('day', event_time)::DATE AS date
  FROM (
    SELECT *
    FROM levels l
    LEFT JOIN installs i
    ON i.id = l.user_id
    WHERE event_time < (install_time + INTERVAL '24 hours')
  ) as sq
  GROUP BY user_id, app_name, DATE_TRUNC('day', event_time)
) as sq2
GROUP BY app_name
ORDER BY average_levels DESC
'''

In [15]:
select_pg(sql)

,app_name,average_levels
0,name_2,2.082707
1,name_1,2.081633
2,name_4,1.911765
3,name_5,1.811475
4,name_3,1.718182


## Задание 2
Оценка AB-теста.

На Casual-проекте был проведен AB-тест, направленный на новых игроков. Он заключался в изменении цены покупки одного из стартовых наборов в магазине (набор включает в себя небольшое количество бустеров, внутриигровой валюты и отключение рекламы). Тестовой группе была предложена цена за набор в 3\$, у контрольной группы она осталась старой - 4\$. После окончания теста были получены такие результаты:


| Группа   | Цена набора | Количество пользователей | Число покупок набора | Средний доход от всех покупок за первые 3 дня жизни игрока (iap arpu 3) |
|----------|-------------|--------------------------|------------------------|-----------------------------------------------------------------------|
| control | \$4          | 10500                    | 180                    | \$0.31                                                                 |
| test     | \$3          | 10450                    | 240                    | \$0.32                                                                 |

\
Аналитик по результатам теста сделал следующий вывод: конверсия в покупку набора увеличилась более чем на 30%, при этом средний доход от покупок игрока также увеличился, следовательно, победила тестовая группа. Согласны ли вы с аналитиком? На что ему следовало еще обратить внимание? Как бы вы планировали и оценивали подобный тест?

Во-первых, не была определена прокси-метрика, которая могла бы дополнительно подтвердить результаты. Например, использование ARPPU помогло бы получить более полное представление о доходе с каждого платящего пользователя.

Также не была определена мощность теста и минимально значимое различие (MDE), что помогло бы определить не только статистическую значимость, но и практическую важность обнаруженных изменений. Аналитик сделал выводы, основываясь на средних значениях, но не провел анализ распределений и не увидел возможные изменения в различных сегментах пользователей.

Необходимо также учитывать потери в других метриках, например, с учетом введения новых наборов по более низкой цене, возможно, мы теряем доход от рекламы из-за уменьшения ее показов пользователям, что также нужно учесть в анализе.

Таким образом, выводы аналитика слишком поверхностные, и необходимо более глубокое исследование, включая анализ не только ключевой метрики, но и других показателей, а также учет практической значимости обнаруженных различий.

План проведения A/B-теста:
* Выбрать основную метрику за которой мы будем наблюдать. Так же стоит выбрать прокси метрику, которая корелирует с основной метрикой и поможет нам понять направление изменения целевой метрики. Можно выделить барьерную метрику, которую нельзя уменьшать. При проведении эксперемента нужно убедиться что мы не влияем на барьерную метрику или влияет положительно. Такой метрикой может быть маржа.
* Сформулировать нулевую и альтернативную гипотезу. В случае конверсии:  
$H_{0}: CR_{test} = CR_{control}$  
$H_{1}: CR_{test} \not = CR_{control}$

* Подобрать релевантную аудиторию для проведения эксперимента.
* Рассчитать длительность эксперемента. Для этого нам нужно узнать у бизнеса какой минимальный прирост метрики мы хотим зафиксировать. Выбрать длительность на основе исторических данных, для которых рассчитанный MDE наиболее близок к минимальному желаемому эффекту, но не превышает его.
* Если позволяет время и ресурсы провести А/А-тест и убедиться что группы разделены правильлно и статистически значимых различий в целевой метрике нет.
* Выбираем метод тестирования. Если целевая метрика конверсия, то можно использовать Z-test для пропорций. Если нам интересно изменение ARPU, используем t-test.
* На основании полученых результатов статистического теста принимаем или отклоняем нулевую гипотезу.
* Сделать вывод понятный заказчику и принять решение о изменении цены покупки набора.


## Задание 3

3. Создать дешборд.

Нужно создать в Tableau дешборд для отслеживания базовых метрик проекта. Он должен содержать таблицу с метриками и график. [Файл с исходными данными для дешборда](https://drive.google.com/file/d/1DX95HXjZaUG4BRP4wJjwuSoh11bRRR7i/view?usp=drive_link)


***Таблица***


*Сегментация в таблице:*


* платформа,
* тип трафика,
* страна,
* дата.

Желательно иметь возможность посмотреть в таблице метрики как суммарно, так и отдельно по каждому сегменту. На примере платформ: у нас есть две платформы, android и ios, нужно иметь возможность видеть любые метрики по двум платформам вместе и по каждой в отдельности.

*Метрики:*

* Installs - установки
* DAU -  среднее количество активных пользователей в день (для подсчета total можно суммировать)
* ARPU, ARPU IAP, ARPU AD - средний доход (revenue), который приносит 1 активный пользователь в день (arpu iap - от покупок,  arpu ad - от рекламы, arpu - суммарно от покупок и от рекламы)
* ARPPU daily - средний доход (revenue), который приносит 1 платящий пользователь (payers) в день
* % PU daily - среднее отношение количества платящих пользователей (payers) к количеству активных пользователей в день (dau).

***График***

График должен представлять динамику во времени двух метрик. Желательно, чтобы метрики можно было выбирать как параметры. Это условие не обязательно, можно придумать свой вариант представления всех метрик в удобном для пользователя виде.
Метрики те же, что и в таблице.
Возможный вариант представления:

***Фильтрация данных на дешборде***

 Должна применяться одновременно и к данным графика и к данным таблицы. Необходимые фильтры:
* платформа,
* тип трафика,
* страна,
* временной период.

Внешний вид графика и таблицы, а также форматы полей и фильтров остаются на ваше усмотрение, настраивайте их так, как вы считаете наиболее удобным для конечного пользователя.


**Визуализация.**

[Ссылка на дашборд в Tableau Public.](https://public.tableau.com/views/gemedevbook/Dashboard2?:language=en-US&:sid=&:display_count=n&:origin=viz_share_link)